# Answer Key to K-Means Quiz
We might want to take a look at the distribution of the Title+Body length feature we used before and instead of using the raw number of words create categories based on this length: short, longer, and super long.

In the questions below, I'll refer to length of the combined `Title` and `Body` fields as Description Length (and by length we mean the number of words when the text is tokenized with pattern=`"\\W"`).

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, count, desc, explode, lit, min, max, split, stddev, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import RegexTokenizer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Creating Features") \
    .getOrCreate()

### Read Dataset

In [3]:
stack_overflow_data = 'Train_onetag_small.json'

In [4]:
df = spark.read.json(stack_overflow_data)
df.persist()

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### Build Description Length Features

In [5]:
df = df.withColumn("Desc", concat(col("Title"), lit(' '), col("Body")))

In [6]:
regexTokenizer = RegexTokenizer(inputCol="Desc", outputCol="words", pattern="\\W")
df = regexTokenizer.transform(df)

In [7]:
body_length = udf(lambda x: len(x), IntegerType())
df = df.withColumn("DescLength", body_length(df.words))

In [8]:
assembler = VectorAssembler(inputCols=["DescLength"], outputCol="DescVec")
df = assembler.transform(df)

In [15]:
number_of_tags = udf(lambda x: len(x.split(" ")), IntegerType())
df = df.withColumn("NumTags", number_of_tags(df.Tags))

# Question 1
How many times greater is the Description Length of the longest question than the Description Length of the shortest question (rounded to the nearest whole number)?

Tip: Don't forget to import Spark SQL's aggregate functions that can operate on DataFrame columns.

In [9]:
df.agg(min("DescLength")).show()

+---------------+
|min(DescLength)|
+---------------+
|             10|
+---------------+



In [10]:
df.agg(max("DescLength")).show()

+---------------+
|max(DescLength)|
+---------------+
|           7532|
+---------------+



# Question 2
What is the mean and standard deviation of the Description length?

In [11]:
df.agg(avg("DescLength"), stddev("DescLength")).show()

+---------------+-----------------------+
|avg(DescLength)|stddev_samp(DescLength)|
+---------------+-----------------------+
|      180.28187|     192.10819533505023|
+---------------+-----------------------+



# Question 3
Let's use K-means to create 5 clusters of Description lengths. Set the random seed to 42 and fit a 5-class K-means model on the Description length column (you can use KMeans().setParams(...) ). What length is the center of the cluster representing the longest questions?

In [12]:
kmeans = KMeans().setParams(featuresCol="DescVec", predictionCol="DescGroup", k=5, seed=42)
model = kmeans.fit(df)
df = model.transform(df)

In [13]:
df.head()

Row(Body="<p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an image apart from checking the file extension using PHP?</p>\n", Id=1, Tags='php image-processing file-upload upload mime-types', Title='How to check if an uploaded file is an image without mime type?', oneTag='php', Desc="How to check if an uploaded file is an image without mime type? <p>I'd like to check if an uploaded file is an image file (e.g png, jpg, jpeg, gif, bmp) or another file. The problem is that I'm using Uploadify to upload the files, which changes the mime type and gives a 'text/octal' or something as the mime type, no matter which file type you upload.</p>\n\n<p>Is there a way to check if the uploaded file is an imag

In [16]:
df.groupby("DescGroup").agg(avg(col("DescLength")), avg(col("NumTags")), count(col("DescLength"))).orderBy("avg(DescLength)").show()

+---------+------------------+------------------+-----------------+
|DescGroup|   avg(DescLength)|      avg(NumTags)|count(DescLength)|
+---------+------------------+------------------+-----------------+
|        0| 96.71484436347646|   2.7442441184785|            63674|
|        4| 241.0267434466191| 3.093549070868367|            28306|
|        2|499.83863263173606|3.2294372294372296|             6699|
|        1|      1074.2109375|3.2864583333333335|             1152|
|        3|2731.0828402366865|  3.42603550295858|              169|
+---------+------------------+------------------+-----------------+

